In [13]:
import pandas as pd
import numpy as np

In [35]:
columns = ['seq_no', 'piece', 'pc', 'next_pc', 'execute_cycle', 'pred_dir', 'taken', 'src_regs',
            'dst_reg', 'mem_va']
history_log_df = pd.read_csv("output/int_0_trace/int_0_trace_history.log", names = columns)
history_log_df['instClass'] = np.where(history_log_df['taken'] == -1, 'load', 'condBranch')

In [44]:
columns = ['seq_no', 'piece', 'pc', 'pred_cycle', 'fetch_cycle','exec_cycle_pred']
pred_history_log_df = pd.read_csv("output/int_0_trace/int_0_trace_pred_history.log", names = columns)

In [45]:
pred_history_log_df.head()

,seq_no,piece,pc,pred_cycle,fetch_cycle,exec_cycle_pred
0,1a,0,8000036c,638,638,1081
1,55,0,80001924,1552,1553,1564
2,d1,0,fffff21b4c80,2753,2754,2764
3,d4,0,fffff21b4cac,2764,2765,2775
4,e7,0,fffff21b4b34,3200,3200,3427


In [46]:
history_log_joined_df = pd.merge(history_log_df, pred_history_log_df, on=['seq_no', 'piece', 'pc'], how='left')

In [56]:
history_log_joined_df['CycWP'] = np.where(\
    history_log_joined_df['taken'] != -1,\
    np.maximum(history_log_joined_df['fetch_cycle'], history_log_joined_df['exec_cycle_pred']) - history_log_joined_df['pred_cycle'],\
    0
)

In [58]:
history_log_joined_df.head(10)

,seq_no,piece,pc,next_pc,execute_cycle,pred_dir,taken,src_regs,dst_reg,mem_va,instClass,pred_cycle,fetch_cycle,exec_cycle_pred,CycWP
0,3,0,800028dc,800028e0,438,0,-1,[31],30,2147533280,load,NaN,NaN,NaN,0.0
1,4,1,800028dc,800028e0,438,0,-1,[31],19,2147533288,load,NaN,NaN,NaN,0.0
2,6,0,800028e4,800028e8,438,0,-1,[31],20,2147533264,load,NaN,NaN,NaN,0.0
3,7,1,800028e4,800028e8,438,0,-1,[31],21,2147533272,load,NaN,NaN,NaN,0.0
4,c,0,80002920,80002924,439,0,-1,[31],30,2147533296,load,NaN,NaN,NaN,0.0
5,d,1,80002920,80002924,439,0,-1,[31],19,2147533304,load,NaN,NaN,NaN,0.0
6,17,0,80000364,80000368,864,0,-1,[29],30,2147487800,load,NaN,NaN,NaN,0.0
7,18,0,80000368,8000036c,1080,0,-1,[30],0,2147488096,load,NaN,NaN,NaN,0.0
8,1b,0,80000370,80000374,1080,0,-1,[30],32,2147488112,load,NaN,NaN,NaN,0.0
9,1a,0,8000036c,80000370,1081,0,0,[0],256,0,condBranch,638.0,638.0,1081.0,443.0


In [77]:
l = []
GHR = ""
GHR_length = 8
history_log_joined_df['GHR'] = 0
# print("HI")
for index,row in history_log_joined_df.iterrows():
    if index > 200:
        break
    if(row['instClass'] == 'condBranch'):
        GHR += str(row['taken'])
        # print(GHR,"jdfk")
        history_log_joined_df.iloc[index,-1] = GHR[-1-GHR_length:-1]
        # for i in range(index-1,-1,-1):
        #     l.append(history_log_df.iloc[i,:])
            # print(history_log_df.iloc[i,:])
pd.DataFrame(l).head()

C:\Users\vgr48\AppData\Local\Temp\ipykernel_94636\1865889121.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  history_log_joined_df.iloc[index,-1] = GHR[-1-GHR_length:-1]


""


In [78]:
GHR

'011101110111010101010101'

In [79]:
history_log_joined_df[history_log_joined_df['taken'] != -1].head(20)

,seq_no,piece,pc,next_pc,execute_cycle,pred_dir,taken,src_regs,dst_reg,mem_va,instClass,pred_cycle,fetch_cycle,exec_cycle_pred,CycWP,GHR
9,1a,0,8000036c,80000370,1081,0,0,[0],256,0,condBranch,638.0,638.0,1081.0,443.0,
50,55,0,80001924,800019cc,1564,0,1,[64],256,0,condBranch,1552.0,1553.0,1564.0,12.0,0
103,d1,0,fffff21b4c80,fffff21b4ca4,2764,0,1,[64],256,0,condBranch,2753.0,2754.0,2764.0,11.0,01
108,d4,0,fffff21b4cac,fffff21b4aec,2775,0,1,[17],256,0,condBranch,2764.0,2765.0,2775.0,11.0,011
116,e7,0,fffff21b4b34,fffff21b4b38,3427,0,0,[64],256,0,condBranch,3200.0,3200.0,3427.0,227.0,0111
130,127,0,fffff21b4c24,fffff21b4c48,3769,0,1,[64],256,0,condBranch,3601.0,3602.0,3769.0,168.0,01110
131,136,0,fffff21b4c80,fffff21b4ca4,3781,1,1,[64],256,0,condBranch,3769.0,3770.0,3781.0,12.0,011101
132,139,0,fffff21b4cac,fffff21b4aec,3781,1,1,[17],256,0,condBranch,3770.0,3771.0,3781.0,11.0,0111011
146,14c,0,fffff21b4b34,fffff21b4b38,3787,0,0,[64],256,0,condBranch,3772.0,3772.0,3787.0,15.0,01110111
147,19e,0,fffff21b4cac,fffff21b4aec,3788,1,1,[17],256,0,condBranch,3777.0,3778.0,3788.0,11.0,11101110


In [16]:
history_log_df[history_log_df['taken'] != -1].head(20)

,seq_no,piece,pc,next_pc,execute_cycle,pred_dir,taken,src_regs,dst_reg,mem_va,instClass
9,1a,0,8000036c,80000370,1081,0,0,[0],256,0,condBranch
50,55,0,80001924,800019cc,1564,0,1,[64],256,0,condBranch
103,d1,0,fffff21b4c80,fffff21b4ca4,2764,0,1,[64],256,0,condBranch
108,d4,0,fffff21b4cac,fffff21b4aec,2775,0,1,[17],256,0,condBranch
116,e7,0,fffff21b4b34,fffff21b4b38,3427,0,0,[64],256,0,condBranch
130,127,0,fffff21b4c24,fffff21b4c48,3769,0,1,[64],256,0,condBranch
131,136,0,fffff21b4c80,fffff21b4ca4,3781,1,1,[64],256,0,condBranch
132,139,0,fffff21b4cac,fffff21b4aec,3781,1,1,[17],256,0,condBranch
146,14c,0,fffff21b4b34,fffff21b4b38,3787,0,0,[64],256,0,condBranch
147,19e,0,fffff21b4cac,fffff21b4aec,3788,1,1,[17],256,0,condBranch
